In [1]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 536.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.3/260.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.2/447.2 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [45]:
from os.path import exists
import pandas as pd

df_path="train.cvs.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=5_000, random_state=0)
df["stores"]= df["store"].astype(str)
df["item"]=df["item"].astype(str)

df["date"]= pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales,stores
0,2013-01-01,5,24,26,5
1,2013-01-01,3,27,11,3
2,2013-01-01,10,21,33,10
3,2013-01-02,1,21,28,1
4,2013-01-02,6,40,9,6


In [46]:
train = df[df["date"]< "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [47]:
train_features= train.drop(columns=["sales"])
train_target= train["sales"]
test_features= test.drop(columns=["sales"])
test_target= test["sales"]

In [48]:
from upgini import FeaturesEnricher , SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])



[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=e3a14f59-8602-4876-a5a9-1f786a10a212
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


37 relevant feature(s) found with the search keys: ['date']


f_events_date_week_sin1_847b5db1,2.1423,100.0000,"0.9749, -0.4339, -0.7818",Upgini,Calendar data,Daily
f_events_date_year_cos1_9014a856,2.0740,100.0000,"-0.9822, -0.8529, -0.505",Upgini,Calendar data,Daily
f_autofe_mul_38705cbc,1.9689,100.0000,"-0.2098, 0.7522, 0.9159",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_b97cf188,1.6626,84.8302,"-1.437, 0.2846, -0.9241",Upgini,AutoFE: features from Calendar data,Daily
f_economic_date_cci_pca_5_b433b3c8,1.6595,100.0000,"-3.1903, -0.9892, 0.361",Upgini,World economic indicators,Daily
f_autofe_mul_4ba12f29,0.4772,100.0000,"6580.6902, 6008.6984, 6664.824...",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_76f25d5b,0.4657,100.0000,"0.9675, 0.9686, 0.9806",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_5feff686,0.4307,100.0000,"-0.9201, -0.1506, -0.9993",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_div_fa4260d8,0.4198,100.0000,"1.0008, 0.9345, 0.9813",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_4a9fa180,0.3978,100.0000,"-0.3005, -0.1853, 0.0496",Upgini,AutoFE: features from Calendar data,Daily
f_financial_date_usd_eur_7d_to_7d_1y_shift_05302176,0.3933,100.0000,"0.9939, 0.9811, 0.9513",Upgini,Markets data,Daily


Upgini,Calendar data,4.2163,2
Upgini,"AutoFE: features from Calendar data,Markets data",3.9904,10
Upgini,AutoFE: features from Markets data,3.4287,14
Upgini,AutoFE: features from Calendar data,3.4097,9
Upgini,World economic indicators,1.6595,1
Upgini,Markets data,0.3933,1


"Calendar data,Markets data",f_autofe_mul_38705cbc,f_events_date_year_cos1_9014a856,f_financial_date_silver_7d_to_1y_0ccfe462,*
Calendar data,f_autofe_div_b97cf188,f_events_date_week_cos3_7525fe31,f_events_date_week_sin2_e28440a5,/
Markets data,f_autofe_mul_4ba12f29,f_financial_date_silver_e4e33014,f_financial_date_stoxx_043cbcd4,*
Markets data,f_autofe_div_76f25d5b,f_financial_date_dow_jones_7d_to_7d_1y_shift_0674410f,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,/
Calendar data,f_autofe_mul_5feff686,f_events_date_year_cos1_9014a856,f_events_date_year_cos2_7acbaf1b,*
Markets data,f_autofe_div_fa4260d8,f_financial_date_dow_jones_7d_to_7d_1y_shift_0674410f,f_financial_date_snp500_7d_to_7d_1y_shift_fa788602,/
Calendar data,f_autofe_mul_4a9fa180,f_events_date_year_cos1_9014a856,f_events_date_year_sin2_59955ffd,*
Markets data,f_autofe_div_8894cba5,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_financial_date_gold_7d_to_1y_ae310379,/
"Calendar data,Markets data",f_autofe_mul_55623469,f_events_date_week_cos3_7525fe31,f_financial_date_gold_1d_to_7d_742a0045,*
"Calendar data,Markets data",f_autofe_div_b1fea8ad,f_events_date_week_cos3_7525fe31,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_mul_31f6ebbe,f_events_date_week_cos3_7525fe31,f_financial_date_stoxx_1d_to_7d_c615714e,*


We detected 26 outliers in your sample.
Examples of outliers with maximum value of target:
10    196
21    176
17    169
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,3975,50.4000,351.434 ± 108.751,272.251 ± 76.154,79.1828
Eval 1,1025,59.7873,570.068 ± 112.008,457.098 ± 143.910,112.9698


In [49]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator= model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


f_events_date_week_sin1_847b5db1,2.2766,100.0000,"0.0, -0.9749, -0.4339",Upgini,Calendar data,Daily
f_autofe_mul_38705cbc,2.2480,100.0000,"0.7286, -0.4024, 0.7517",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_economic_date_cci_pca_5_b433b3c8,1.6791,100.0000,"-2.1092, -2.5687, -1.5988",Upgini,World economic indicators,Daily
f_events_date_year_cos1_9014a856,1.5982,100.0000,"-0.6839, 0.9413, -0.9955",Upgini,Calendar data,Daily
f_autofe_div_b97cf188,1.5959,84.8302,"0.9241, 0.2846, 1.437",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_div_fa4260d8,0.6879,100.0000,"0.9807, 0.9862, 0.9792",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_4a9fa180,0.6779,100.0000,"0.4722, 0.7642, 0.7663",Upgini,AutoFE: features from Calendar data,Daily
f_autofe_div_8894cba5,0.6062,100.0000,"0.6102, 0.5081, 1.1142",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_4ba12f29,0.5652,100.0000,"7289.7005, 6466.9683, 6619.113",Upgini,AutoFE: features from Markets data,Daily
f_autofe_div_c689dfb7,0.5498,100.0000,"0.9575, 0.9004, 0.9743",Upgini,AutoFE: features from Markets data,Daily
f_autofe_mul_5feff686,0.4909,100.0000,"0.82, -0.2434, -0.2678",Upgini,AutoFE: features from Calendar data,Daily


Upgini,AutoFE: features from Markets data,5.3642,14
Upgini,"AutoFE: features from Calendar data,Markets data",4.8765,10
Upgini,AutoFE: features from Calendar data,4.4580,9
Upgini,Calendar data,3.8748,2
Upgini,World economic indicators,1.6791,1
Upgini,Markets data,0.3275,1


"Calendar data,Markets data",f_autofe_mul_38705cbc,f_events_date_year_cos1_9014a856,f_financial_date_silver_7d_to_1y_0ccfe462,*
Calendar data,f_autofe_div_b97cf188,f_events_date_week_cos3_7525fe31,f_events_date_week_sin2_e28440a5,/
Markets data,f_autofe_div_fa4260d8,f_financial_date_dow_jones_7d_to_7d_1y_shift_0674410f,f_financial_date_snp500_7d_to_7d_1y_shift_fa788602,/
Calendar data,f_autofe_mul_4a9fa180,f_events_date_year_cos1_9014a856,f_events_date_year_sin2_59955ffd,*
Markets data,f_autofe_div_8894cba5,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_financial_date_gold_7d_to_1y_ae310379,/
Markets data,f_autofe_mul_4ba12f29,f_financial_date_silver_e4e33014,f_financial_date_stoxx_043cbcd4,*
Markets data,f_autofe_div_c689dfb7,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_stoxx_7d_to_7d_1y_shift_abef6e91,/
Calendar data,f_autofe_mul_5feff686,f_events_date_year_cos1_9014a856,f_events_date_year_cos2_7acbaf1b,*
Markets data,f_autofe_div_76f25d5b,f_financial_date_dow_jones_7d_to_7d_1y_shift_0674410f,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,/
Calendar data,f_autofe_div_0a9c9355,f_events_date_month_cos3_a9200432,f_events_date_week_sin2_e28440a5,/
Markets data,f_autofe_mul_ba63ade6,f_financial_date_crude_oil_1d_to_7d_471b3b15,f_financial_date_usd_7d_to_7d_1y_shift_35cb53e9,*


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,3975,50.4000,0.263 ± 0.033,0.194 ± 0.052,0.068550
1,Eval 1,1025,59.7873,0.272 ± 0.017,0.205 ± 0.038,0.066902


In [50]:
enriched_train_features= pd.DataFrame(enricher.transform(train_features, keep_input=True))
enriched_test_features= pd.DataFrame(enricher.transform(test_features, keep_input=True))
enriched_train_features.head()

[============================================================] 100% Finished

You are trying to launch enrichment for 3975 rows, which will exceed the rest limit 1000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You are trying to launch enrichment for 1025 rows, which will exceed the rest limit 1000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

""


In [16]:
model.fit(train_features, train_target)
preds= model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.254216275133324]